In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime
import time
import os

In [39]:
# Pasta dos arquivos
pasta = '/Users/bentofh/Documents/GitHub/ancine/arquivos_parquet/'

In [40]:
# Importando arquivo
df = pd.read_parquet(os.path.join(pasta,r'BrilheteriaAmostra.parquet'))
df.shape

(10000, 8)

In [42]:
# Extrair os títulos únicos e normalizar para o formato de pesquisa
titulos_unicos = df['TITULO_ORIGINAL'].str.title().dropna().unique()
len(titulos_unicos)

115

In [45]:
# Extrair os títulos únicos e normalizar para o formato de pesquisa
titulos_unicos = df['TITULO_ORIGINAL'].str.title().dropna().unique()

def buscar_info_filme(titulo, api_key):
    try:
        url = f"http://www.omdbapi.com/?t={titulo}&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        if data['Response'] == 'True':
            return data
        else:
            return None
    except:
        return None

# Sua chave API da OMDB
api_key = '6af69e78'

# Dicionário para armazenar os dados
dados_filmes = []

# Buscar informações para cada título
for titulo in titulos_unicos:
    info_filme = buscar_info_filme(titulo, api_key)
    if info_filme:
        # Adicionar a data e hora da coleta
        info_filme['Data_Coleta'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        dados_filmes.append(info_filme)
    time.sleep(0.5)  # Pausa para evitar sobrecarregar a API

# Criar DataFrame com os dados coletados
df_info_filmes = pd.DataFrame(dados_filmes)


# Criar nova coluna com apenas o primeiro gênero
df_info_filmes['PrimeiroGenero'] = df_info_filmes['Genre'].apply(lambda x: x.split(',')[0] if pd.notnull(x) else x)

# Reordenar as colunas para que 'Data_Coleta' seja a primeira
colunas = ['Data_Coleta'] + [col for col in df_info_filmes if col != 'Data_Coleta']
df_info_filmes = df_info_filmes[colunas]

# Exportar para CSV
df_info_filmes.to_csv(os.path.join(pasta, 'informacoes_filmes.csv'), index=False)

# Criar nova coluna com apenas o primeiro gênero
df_info_filmes['PrimeiroGenero'] = df_info_filmes['Genre'].apply(lambda x: x.split(',')[0] if pd.notnull(x) else x)

# Reordenar as colunas para que 'Data_Coleta' seja a primeira
colunas = ['Data_Coleta'] + [col for col in df_info_filmes if col != 'Data_Coleta']
df_info_filmes = df_info_filmes[colunas]

# Exportar para CSV
df_info_filmes.to_csv(os.path.join(pasta, 'informacoes_filmes.csv'), index=False)

In [49]:
# Arquivos com as informações recuperadas na API
df_info_filmes.head(5)

,Data_Coleta,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,2024-01-18 14:14:18,PUSS IN BOOTS: THE LAST WISH,2022,PG,21 Dec 2022,102 min,"Animation, Adventure, Comedy","Joel Crawford, Januel Mercado","Paul Fisher, Tommy Swerdlow, Tom Wheeler","Antonio Banderas, Salma Hayek, Harvey Guillén",...,7.8,"163,130",tt3915174,movie,NaN,"$185,535,345",20th Century Fox,NaN,True,NaN
1,2024-01-18 14:14:19,ANT-MAN AND THE WASP: QUANTUMANIA,2023,PG-13,17 Feb 2023,124 min,"Action, Adventure, Comedy",Peyton Reed,"Jeff Loveness, Stan Lee, Larry Lieber","Paul Rudd, Evangeline Lilly, Michael Douglas",...,6.1,"214,985",tt10954600,movie,NaN,"$214,504,909",NaN,NaN,True,NaN
2,2024-01-18 14:14:20,THE SUPER MARIO BROS. MOVIE,2023,PG,05 Apr 2023,92 min,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic, Pierre Leduc",Matthew Fogel,"Chris Pratt, Anya Taylor-Joy, Charlie Day",...,7.0,"224,247",tt6718170,movie,16 May 2023,"$574,934,330",NaN,NaN,True,NaN
3,2024-01-18 14:14:22,EVIL DEAD RISE,2023,R,21 Apr 2023,96 min,Horror,Lee Cronin,Lee Cronin,"Mirabai Pease, Richard Crouchley, Anna-Maree T...",...,6.5,"129,628",tt13345606,movie,09 May 2023,"$67,233,054",NaN,NaN,True,NaN
4,2024-01-18 14:14:23,KNIGHTS OF THE ZODIAC,2023,PG-13,12 May 2023,112 min,"Action, Adventure, Drama",Tomasz Baginski,"Josh Campbell, Matt Stuecken, Kiel Murray","Mackenyu, Famke Janssen, Madison Iseman",...,4.4,"11,510",tt6528290,movie,NaN,"$1,090,155",NaN,NaN,True,NaN


In [57]:
# Carregar o DataFrame com as informações dos filmes obtidas da API
df_info_filmes = pd.read_csv(os.path.join(pasta,r'informacoes_filmes.csv'))

# Converter os títulos para um formato comum (por exemplo, maiúsculas) para garantir a correspondência correta
df['TITULO_ORIGINAL'] = df['TITULO_ORIGINAL'].str.upper()
df_info_filmes['Title'] = df_info_filmes['Title'].str.upper()

# Fazer o merge dos DataFrames com base no título do filme
df_merged = pd.merge(df, df_info_filmes, left_on='TITULO_ORIGINAL', right_on='Title', how='left')

# Agora você pode analisar os gêneros de filmes
# Por exemplo, contar quantas vezes cada gênero aparece
generos = df_merged['PrimeiroGenero'].str.split(', ').explode()
contagem_generos = generos.value_counts()

print(contagem_generos)

PrimeiroGenero
Action         3831
Animation      3460
Horror         1117
Drama           607
Comedy          565
Biography        29
Documentary      19
Short            10
Adventure         8
Crime             6
Name: count, dtype: int64
